In [1]:
import numpy as np
import math
import pandas_datareader as pdr
#import matplotlib.pyplot as plt
#%matplotlib inline
import datetime
import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like

import yfinance as yf
yf.pdr_override()
from pandas import Series, DataFrame

In [2]:
fileName = 'test.csv'
start = datetime.date(2019, 1, 1)#resolve if only input year later
end = datetime.date(2019, 2, 1)
if fileName[fileName.rfind(".")+1:] == 'csv':
     df = pd.read_csv(fileName)
elif fileName[fileName.rfind(".")+1:] == 'xlsx':
     df = pd.read_excel(fileName)
else : 
    print('Wrong File Type')

df.dropna(axis=0,how='any') #drop all rows that have any NaN values
Names = df.iloc[:,0] #first column

tickerList = Names.array #convert Series to array

#retrieve data from yahoo finance
all_data = {}
for ticker in tickerList:
    all_data[ticker] = pdr.get_data_yahoo(ticker, start, end)

#store data in DataFrame
price = DataFrame({tic: data['Adj Close']
                    for tic, data in all_data.items()})
print (price)

                   SPY        AAPL
Date                              
2019-01-02  247.877258  156.642365
2019-01-03  241.962219  141.039642
2019-01-04  250.066925  147.060516
2019-01-07  252.038620  146.733185
2019-01-08  254.406601  149.530380
2019-01-09  255.595551  152.069672
2019-01-10  256.497192  152.555710
2019-01-11  256.596252  151.057922
2019-01-14  255.030792  148.786438
2019-01-15  257.953644  151.831619
2019-01-16  258.577881  153.686493
2019-01-17  260.539612  154.599045
2019-01-18  264.007385  155.551285
2019-01-22  260.440552  152.059753
2019-01-23  260.985504  152.674728
2019-01-24  261.124207  151.464615
2019-01-25  263.333679  156.483673
2019-01-28  261.332275  155.035492
2019-01-29  260.985504  153.428589
2019-01-30  265.117096  163.913071
2019-01-31  267.445465  165.093445
2019-02-01  267.574280  165.172806


In [3]:
Return = price.shift(-1)/price

In [4]:
#Return.dropna(axis=0,how='any')
Return.drop(Return.tail(1).index,inplace=True) #drop the last row, NaN
LogReturn = np.log(Return.to_numpy().transpose())
print(LogReturn)

[[-0.0241521   0.03294699  0.00785375  0.00935145  0.00466254  0.0035214
   0.00038613 -0.00611956  0.0113956   0.00241703  0.00755798  0.01322217
  -0.01360245  0.00209024  0.00053132  0.00842579 -0.00762929 -0.00132782
   0.01570673  0.00874408  0.00048153]
 [-0.10492427  0.04180318 -0.00222831  0.01888372  0.0168392   0.00319106
  -0.00986648 -0.01515138  0.02026016  0.01214263  0.00592019  0.00614053
  -0.02270192  0.00403614 -0.00795767  0.03259965 -0.00929761 -0.01041883
   0.06610099  0.00717542  0.00048059]]


In [5]:
#arithmetic mean of historical log return
mean_his = np.mean(LogReturn,axis = 1) #average of each row
print(mean_his)

[0.00364112 0.00252509]


In [6]:
std = np.std(LogReturn, axis = 1) #standard deviation
print(std)

[0.01117601 0.0312512 ]


In [9]:
#get current US 10 year Treasury rate from web
import urllib
import requests
requests.packages.urllib3.disable_warnings()
from bs4 import BeautifulSoup
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context
url = "https://www.treasury.gov/resource-center/data-chart-center/interest-rates/Pages/TextView.aspx"
#url = "https://www.cnbc.com/quotes"
#url = "https://www.bloomberg.com/markets/rates-bonds/government-bonds/us"
params = {"data":"yield"}
headers = {}
headers['User-Agent'] = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.142 Safari/537.36"
req = urllib.request.Request(url,headers = headers,data=urllib.parse.urlencode(params).encode("utf-8"))
response = urllib.request.urlopen(req)
soup = BeautifulSoup(response)
print(soup.prettify())


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html __expr-val-dir="ltr" dir="ltr" lang="en-us" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="0" http-equiv="Expires"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   Text View
  </title>
  <link href="/Style%20Library/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
  <link href="/_layouts/1033/styles/Themable/Menu.css?rev=j46FAsDJga%2BaixwEf8A%2Fgg%3D%3D" rel="stylesheet" type="text/css"/>
  <link href="/_layouts/1033/styles/Themable/corev4.css?rev=p63%2BuzTeSJc22nVGNZ5zwg%3D%3D" rel="stylesheet" type="text/css"/>
  <link href="/Style%20Library/jquery-ui-1.8.6.custom/css/ui-lightness/jquery-ui-exit.css" rel="stylesheet" type="text/css"/>
  <link href="/Style%20Library/custom/webEditing.css" rel="stylesheet" type="text/css"/>
  <link href="/Style%2

In [13]:
#s = soup.find(type="text/javascript").contents[0].string
#index = s.find("last")+len("last")+3
#rf = float(s[index:index+5])
#print(rf)
print(soup.find("1.92"))

None


In [20]:
import scipy.optimize as scipy
#construct an equal weight
length = len(tickerList)
w = np.full(length,1/length) #initialize an array with fill_value=1/length, and shape = length
#mean of historical return of the portfolio if equal weights
mean_his_pf = mean_his * w
print(mean_his_pf)
#def objective(x) :
    

[0.00182056 0.00126255]
